N: 공정 수

W: width(m)

W = 140

Hbay = 25

N = 10 데이터


In [2]:
import numpy as np
import pandas as pd

# 공정 이름
processes = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']

# 공정 너비 (m)
widths = {
    'D1': 6.9,
    'D2': 11.8,
    'D3': 28.0,
    'D4': 20.7,
    'D5': 61.5,
    'D6': 15.2,
    'D7': 6.5,
    'D8': 20.8,
    'D9': 5.4,
    'D10': 74.2
}

# Flow density matrix (From-To Matrix)
flow_matrix = np.array([
    [0, 1, 7, 1, 1, 0, 1, 2, 1, 0],
    [2, 0, 4, 0, 0, 0, 5, 1, 0, 2],
    [10, 10, 0, 0, 6, 3, 1, 9, 2, 5],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 8],
    [1, 1, 2, 0, 0, 1, 5, 3, 2, 0],
    [0, 0, 0, 0, 1, 0, 5, 0, 5, 0],
    [0, 1, 15, 0, 1, 1, 0, 3, 0, 2],
    [0, 0, 6, 0, 1, 0, 0, 1, 0, 15],
    [2, 0, 1, 0, 3, 2, 1, 3, 1, 0],
    [0, 0, 12, 1, 3, 1, 5, 3, 1, 0]
])

# Fab 관련 설정
W = 140  # W
Hbay = 25       # H_bay
Hcc = 10        # H_cc (중앙 복도 높이, 논문에는 명시 안됨, 임의값 설정 가능)

# 데이터프레임 정리
process_df = pd.DataFrame({
    'Process': processes,
    'Width (m)': [widths[p] for p in processes]
})

# Flow matrix를 데이터프레임으로 변환
flow_df = pd.DataFrame(flow_matrix, index=processes, columns=processes)

process_df


,Process,Width (m)
0,D1,6.9
1,D2,11.8
2,D3,28.0
3,D4,20.7
4,D5,61.5
5,D6,15.2
6,D7,6.5
7,D8,20.8
8,D9,5.4
9,D10,74.2


In [3]:
flow_df

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10
D1,0,1,7,1,1,0,1,2,1,0
D2,2,0,4,0,0,0,5,1,0,2
D3,10,10,0,0,6,3,1,9,2,5
D4,0,0,0,0,0,0,0,0,0,8
D5,1,1,2,0,0,1,5,3,2,0
D6,0,0,0,0,1,0,5,0,5,0
D7,0,1,15,0,1,1,0,3,0,2
D8,0,0,6,0,1,0,0,1,0,15
D9,2,0,1,0,3,2,1,3,1,0
D10,0,0,12,1,3,1,5,3,1,0


In [5]:
import numpy as np
import random
from copy import deepcopy

class FabGA:
    def __init__(self, processes, flow_matrix, widths, W, Hbay, Hcc,
                 weights=(1,5,5), pop_size=200, rc=0.9, rm=0.05):
        """
        processes: list of process names
        flow_matrix: NxN numpy array of flow densities
        widths: dict mapping process to width
        W: Fab total width
        Hbay: bay height
        Hcc: central corridor height
        weights: (w1, w2, w3) for forward, reverse, cross
        pop_size: population size Q
        rc: crossover rate
        rm: mutation rate
        """
        self.processes = processes
        self.flow = flow_matrix
        self.widths = widths
        self.W = W
        self.Hbay = Hbay
        self.Hcc = Hcc
        self.upper_y = Hbay + Hcc + Hbay/2
        self.lower_y = Hbay/2
        self.w1, self.w2, self.w3 = weights
        self.pop_size = pop_size
        self.rc = rc
        self.rm = rm
        self.pop = []
        self.fitness = []

    def generate_initial_population(self):
        """Random permutations of processes"""
        self.pop = [random.sample(self.processes, len(self.processes))
                    for _ in range(self.pop_size)]

    def is_feasible(self, chrom):
        """Check bay width constraints"""
        mid = len(chrom)//2
        upper_w = sum(self.widths[p] for p in chrom[:mid])
        lower_w = sum(self.widths[p] for p in chrom[mid:])
        return upper_w <= self.W and lower_w <= self.W

    def compute_coordinates(self, chrom):
        """Compute (x,y) for each process"""
        mid = len(chrom)//2
        upper = chrom[:mid]
        lower = chrom[mid:]
        coords = {}
        x = 0
        for p in upper:
            w = self.widths[p]
            coords[p] = (x + w/2, self.upper_y)
            x += w
        x = 0
        for p in reversed(lower):
            w = self.widths[p]
            coords[p] = (x + w/2, self.lower_y)
            x += w
        return coords

    def evaluate_fitness(self, chrom):
        """Compute Fval and ConDeg for chromosome"""
        coords = self.compute_coordinates(chrom)
        n = len(chrom)
        mid = len(chrom)//2
        Fit1 = Fit2 = Fit3 = 0.0
        # distances and flow classification
        for i, pi in enumerate(chrom):
            x1, y1 = coords[pi]
            for j, pj in enumerate(chrom):
                if i == j: continue
                f = self.flow[self.processes.index(pi), self.processes.index(pj)]
                if f == 0: continue
                x2, y2 = coords[pj]
                d = abs(x1 - x2) + abs(y1 - y2)
                
                if y1 == y2 == self.upper_y:
                    if x2 > x1:
                        Fit1 += f * d
                    else:
                        Fit2 += f * d
                elif y1 == y2 == self.lower_y:
                    if x2 < x1:
                        Fit1 += f * d
                    else:
                        Fit2 += f * d
                else:
                    Fit3 += f * d
        # penalty if infeasible
        Fit4 = 0.0
        if not self.is_feasible(chrom):
            upper_w = sum(self.widths[p] for p in chrom[:mid])
            lower_w = sum(self.widths[p] for p in chrom[mid:])
            over_upper = max(0, upper_w - W)
            over_lower = max(0, lower_w - W)
            Fit4 = over_upper + over_lower
        Fval = self.w1*Fit1 + self.w2*Fit2 + self.w3*Fit3 + Fit4
        ConDeg = (self.w2*Fit2 + self.w3*Fit3) / (Fval)
        return Fval, ConDeg

    def selection(self):
        """Tournament selection"""
        selected = []
        for _ in range(self.pop_size):
            a, b = random.sample(range(self.pop_size), 2)
            # lower Fval is better
            if self.fitness[a][0] < self.fitness[b][0]:
                selected.append(deepcopy(self.pop[a]))
            else:
                selected.append(deepcopy(self.pop[b]))
        self.pop = selected

    def crossover(self, parent1, parent2, method='1point'):
        """One-point or two-point crossover without duplicates"""
        n = len(parent1)
        if method == '1point':
            i = random.randrange(n)
            seq = [p for p in parent2 if parent1.index(p) >= i]
            child = parent1[:i] + seq
        else:  # two-point
            i, j = sorted(random.sample(range(n), 2))
            seq = [p for p in parent2 if parent1.index(p) >= i and parent1.index(p) < j]
            child = parent1[:i] + seq + parent1[j:]
        return child

    def mutate(self, chrom):
        """Swap mutation"""
        if random.random() < self.rm:
            i, j = random.sample(range(len(chrom)), 2)
            chrom[i], chrom[j] = chrom[j], chrom[i]

    def run(self, generations=300, patience=50, min_delta=1e-6):
        """Main GA loop with early stopping based on no improvement"""
        # initialize
        self.generate_initial_population()
        # evaluate initial
        self.fitness = [self.evaluate_fitness(c) for c in self.pop]
        best = min(self.fitness, key=lambda x: x[0])
        best_chrom = self.pop[self.fitness.index(best)]
        no_improve = 0
        # evolution
        for gen in range(generations):
            self.selection()
            next_pop = []
            for i in range(0, self.pop_size, 2):
                p1, p2 = self.pop[i], self.pop[i+1]
                if random.random() < self.rc:
                    c1 = self.crossover(p1, p2, '1point')
                    c2 = self.crossover(p2, p1, '1point')
                else:
                    c1, c2 = deepcopy(p1), deepcopy(p2)
                self.mutate(c1); self.mutate(c2)
                next_pop += [c1, c2]
            self.pop = next_pop
            self.fitness = [self.evaluate_fitness(c) for c in self.pop]
            current_best = min(self.fitness, key=lambda x: x[0])
            
             # Check for improvement
            if best[0] - current_best[0] > min_delta:
                best, best_chrom = current_best, self.pop[self.fitness.index(current_best)]
                no_improve = 0
            else:
                no_improve += 1
            # Early stopping if no improvement
            if no_improve >= patience:
                break
            

        return best_chrom, best


In [10]:
# 1) 필요한 데이터 불러오기
#processes = ['D1','D2',...,'D10']
#flow_matrix = np.array([...])  # Figure 3 데이터
#widths = {...}                # Table 2 데이터

# 2) GA 객체 생성
ga = FabGA(processes, flow_matrix, widths,
           W=140, Hbay=25, Hcc=10,
           weights=(1,5,5),
           pop_size=200, rc=0.9, rm=0.05)
start = time.time()
# 3) GA 실행 
best_layout, best_metrics = ga.run(generations=500, patience=50, min_delta=1e-6)
end = time.time()
print("최적 배치:", best_layout)
print("최적 Fval, ConDeg:", best_metrics)
print(f"걸린 시간: {end - start:.1f}초")

최적 배치: ['D2', 'D3', 'D7', 'D8', 'D10', 'D4', 'D5', 'D1', 'D6', 'D9']
최적 Fval, ConDeg: (np.float64(34422.8), np.float64(0.9320058217228115))
걸린 시간: 2.1초


In [9]:
import itertools
import time

# 브루트포스 탐색
best_Fval = float('inf')
best_chrom = None

start = time.time()
for chrom in itertools.permutations(processes):
    Fval, ConDeg = ga.evaluate_fitness(chrom)
    if Fval < best_Fval:
        best_Fval = Fval
        best_ConDeg = ConDeg
        best_chrom = chrom
end = time.time()

print("Brute-force 최적 해:", best_chrom)
print("Brute-force 최적 Fval:", best_Fval)
print("Brute-force 최적 ConDeg:", best_ConDeg)
print(f"걸린 시간: {end - start:.1f}초")


Brute-force 최적 해: ('D6', 'D9', 'D1', 'D5', 'D4', 'D10', 'D8', 'D7', 'D3', 'D2')
Brute-force 최적 Fval: 33230.4
Brute-force 최적 ConDeg: 0.9273060209928258
걸린 시간: 560.5초


똑같이 나왔다. 시간은 엄청 더 빠르다.